<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/02_prompt_engineering.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Prompt Engineering

## Overview  
In this exercise, we will explore prompt engineering within the Solar framework. Prompt engineering is a crucial technique in leveraging large language models effectively by crafting prompts that elicit the desired responses from the model. This tutorial will walk you through various strategies for creating and refining prompts to optimize the performance of Solar in different NLP tasks.

## Purpose of the Exercise
The purpose of this exercise is to equip users with practical skills in prompt engineering. By the end of this tutorial, users will be able to design effective prompts, understand the impact of prompt variations, and enhance the accuracy and relevance of responses generated by the Solar LLM. This foundational knowledge is essential for advanced applications and fine-tuning of language models.


In [1]:
! pip install -qU langchain-upstage python-dotenv

In [6]:
# @title set API key
# First, enroll your API key as the colab key.
from pprint import pprint
import os

import warnings

warnings.filterwarnings("ignore")

from IPython import get_ipython

upstage_api_key_env_name = "upstage_api_key"


def load_env():
    if "google.colab" in str(get_ipython()):
        # Running in Google Colab
        from google.colab import userdata

        upstage_api_key = userdata.get(upstage_api_key_env_name)
        # print(upstage_api_key)
        return os.environ.setdefault(upstage_api_key_env_name, upstage_api_key)
    else:
        # Running in local Jupyter Notebook
        from dotenv import load_dotenv

        load_dotenv()
        return os.environ.get(upstage_api_key_env_name)


UPSTAGE_API_KEY = load_env() # Setting API Key

In [7]:
# Quick hello world
from langchain_upstage import ChatUpstage

llm = ChatUpstage(api_key=UPSTAGE_API_KEY)
response = llm.invoke("What is the capital of France")

print(response)
print(response.content)

content='The capital of France is Paris.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 16, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-mini-250422', 'system_fingerprint': None, 'id': '9b312838-8b63-4bdd-918c-8ec60f68e521', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--f3f156c7-fe30-4eea-90e5-93a9d62d4bf3-0' usage_metadata={'input_tokens': 16, 'output_tokens': 8, 'total_tokens': 24, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
The capital of France is Paris.


In [8]:
# Chat prompt
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "What is the capital of France?"),
        ("ai", "Answer: It's Paris!!"),
        ("human", "What about Korea?"),
    ]
)

# 3. define chain
from langchain_core.output_parsers import StrOutputParser

chain = chat_prompt | llm
response = chain.invoke({})

print(response.content)

Answer: It's Seoul!!


In [9]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: The cafeteria had 23 apples.
If they used 20 to make lunch and bought 6 more,
how many apples do they have?

A: the answer is
"""
)
chain = prompt_template | llm
response = chain.invoke({})

print(response.content)

To solve this problem, let's break it down step by step:

1. The cafeteria initially had 23 apples.
2. They used 20 apples to make lunch, so we subtract those from the initial amount:
   \[
   23 - 20 = 3
   \]
   After making lunch, they have 3 apples left.

3. They then bought 6 more apples, so we add those to the remaining apples:
   \[
   3 + 6 = 9
   \]

Therefore, the cafeteria has 9 apples in total.


In [10]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

A: The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

A: the answer is
"""
)
chain = prompt_template | llm
response = chain.invoke({})

print(response.content)

To solve this problem, we need to calculate the total number of apples the cafeteria has after using some for lunch and buying more.

1. The cafeteria started with 23 apples.
2. They used 20 apples to make lunch, so we subtract those from the initial amount:  
   \( 23 - 20 = 3 \) apples remaining.
3. They then bought 6 more apples, so we add those to the remaining apples:  
   \( 3 + 6 = 9 \) apples.

Therefore, the cafeteria has 9 apples now.


![CoT](figures/cot.webp)

from https://arxiv.org/abs/2201.11903

In [11]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

A: Roger started with 5 balls. 2 cans of 3 tennis balls
each is 6 tennis balls. 5 + 6 = 11. The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"""
)
chain = prompt_template | llm
response = chain.invoke({})

print(response.content)

The cafeteria started with 23 apples. They used 20 to make lunch, so they had 23 - 20 = 3 apples left. Then they bought 6 more apples, so they have 3 + 6 = 9 apples now. The answer is 9.


![Zero-Shot COT](figures/zero-cot.webp)

From https://arxiv.org/abs/2205.11916

In [12]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

A: The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

A: Let's think step by step.
"""
)
chain = prompt_template | llm
response = chain.invoke({})

print(response.content)

Step 1: The cafeteria initially had 23 apples.

Step 2: They used 20 apples to make lunch, so we subtract 20 from 23.
23 - 20 = 3

Step 3: They then bought 6 more apples, so we add 6 to the remaining apples.
3 + 6 = 9

Therefore, the cafeteria now has 9 apples.


## divide and conquer
![divideandconquer](figures/divideandconquer.png)

In [13]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide three questions from the following text:
    ---
    We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters,
    demonstrating superior performance in various natural language processing (NLP) tasks.
    Inspired by recent efforts to efficiently up-scale LLMs,
    we present a method for scaling LLMs called depth up-scaling (DUS),
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts,
    DUS does not require complex changes to train and inference efficiently.
    We show experimentally that DUS is simple yet effective
    in scaling up high-performance LLMs from small ones.
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct,
    a variant fine-tuned for instruction-following capabilities,
    surpassing Mixtral-8x7B-Instruct.
    SOLAR 10.7B is publicly available under the Apache 2.0 license,
    promoting broad access and application in the LLM field.
    """
)
chain = prompt_template | llm
response = chain.invoke({})

print(response.content)


1. What is the name of the large language model introduced in the text, and how many parameters does it have?
2. What is the method for scaling LLMs called, and how does it differ from other LLM up-scaling methods?
3. What is the name of the variant of the SOLAR model that has been fine-tuned for instruction-following capabilities, and how does it compare to Mixtral-8x7B-Instruct?


In [14]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please extract three keywords from the following text:
    ---
    We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters,
    demonstrating superior performance in various natural language processing (NLP) tasks.
    Inspired by recent efforts to efficiently up-scale LLMs,
    we present a method for scaling LLMs called depth up-scaling (DUS),
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts,
    DUS does not require complex changes to train and inference efficiently.
    We show experimentally that DUS is simple yet effective
    in scaling up high-performance LLMs from small ones.
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct,
    a variant fine-tuned for instruction-following capabilities,
    surpassing Mixtral-8x7B-Instruct.
    SOLAR 10.7B is publicly available under the Apache 2.0 license,
    promoting broad access and application in the LLM field.
    """
)
chain = prompt_template | llm
response = chain.invoke({})

print(response.content)

1. Large Language Model (LLM)
2. Depth Up-scaling (DUS)
3. Instruction-following capabilities


In [15]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide one question from the following text
    regarding "Depth up-scaling (DUS)":

    ---
    We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters,
    demonstrating superior performance in various natural language processing (NLP) tasks.
    Inspired by recent efforts to efficiently up-scale LLMs,
    we present a method for scaling LLMs called depth up-scaling (DUS),
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts,
    DUS does not require complex changes to train and inference efficiently.
    We show experimentally that DUS is simple yet effective
    in scaling up high-performance LLMs from small ones.
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct,
    a variant fine-tuned for instruction-following capabilities,
    surpassing Mixtral-8x7B-Instruct.
    SOLAR 10.7B is publicly available under the Apache 2.0 license,
    promoting broad access and application in the LLM field.
    """
)
chain = prompt_template | llm
response = chain.invoke({})

print(response.content)

What is the method for scaling large language models (LLMs) introduced in the text, and how does it differ from other LLM up-scaling methods?


In [21]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}. Provide the joke only."
)
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens. Provide the joke only.'

In [22]:
chain = prompt_template | llm | StrOutputParser()
chain.invoke({"adjective": "funny", "content": "chickens"})

"Why don't chickens ever play cards in the jungle?\n\nBecause they're afraid of getting decked!"

In [23]:
chain.invoke({"adjective": "funny", "content": "beef"})

'Why did the steak go to the doctor? Because it felt a little "meaty"!'

In [24]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide one question from the following text
    regarding "{keyword}":

    ---
    {text}
    """
)
chain = prompt_template | llm | StrOutputParser()
keyword = "DUS"
text = """
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters,
    demonstrating superior performance in various natural language processing (NLP) tasks.
    Inspired by recent efforts to efficiently up-scale LLMs,
    we present a method for scaling LLMs called depth up-scaling (DUS),
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts,
    DUS does not require complex changes to train and inference efficiently.
    We show experimentally that DUS is simple yet effective
    in scaling up high-performance LLMs from small ones.
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct,
    a variant fine-tuned for instruction-following capabilities,
    surpassing Mixtral-8x7B-Instruct.
    SOLAR 10.7B is publicly available under the Apache 2.0 license,
    promoting broad access and application in the LLM field.
"""
chain.invoke({"keyword": keyword, "text": text})

'What is the name of the method for scaling large language models introduced in the text, which stands for depthwise scaling and continued pretraining?'

# References
* https://platform.openai.com/docs/guides/prompt-engineering
* https://docs.anthropic.com/claude/docs/intro-to-prompting
* https://smith.langchain.com/hub